In [246]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
from sklearn.cluster import KMeans

In [ ]:
# something like this tog et data from DataStore
#     ds = DataStore(debug=True)
#     print(ds.data.head())
#     print(ds.wifi)
#     ds.data.to_csv('data.csv', sep='\t', encoding='utf-8')

In [247]:
data = pd.read_csv('/Users/nikanizharadze/Desktop/TUM_ScienceHack21-main/model/data.csv', sep = '\t')
wifi = pd.read_csv('/Users/nikanizharadze/Desktop/TUM_ScienceHack21-main/model/wifi.csv', sep='\t')

In [248]:
data.head()

,Unnamed: 0,is_valid,latitude,longitude,speed_over_ground_knots,heading_true_course,epoch_ts,line,bus,Number_of_Passengers
0,0,False,0.000000,0.000000,NaN,NaN,2021-04-12 09:22:22,trip_1,bus_trip_1,-1.0
1,1,True,48.151783,11.525906,19.488,260.93,2021-04-12 09:22:23,trip_1,bus_trip_1,-1.0
2,2,True,48.151754,11.525763,19.709,259.68,2021-04-12 09:22:24,trip_1,bus_trip_1,-1.0
3,3,True,48.151737,11.525618,19.987,260.13,2021-04-12 09:22:25,trip_1,bus_trip_1,-1.0
4,4,True,48.151743,11.525471,20.009,263.38,2021-04-12 09:22:26,trip_1,bus_trip_1,-1.0


In [249]:
wifi['epoch_ts'] = wifi['epoch_ts'].str.split('.').str[0]

In [250]:
df = pd.merge(wifi, data, on=['epoch_ts'])

In [251]:
del df['id']
del df['vendor']
del df['subtype']
del df['Unnamed: 0_x']
del df['Unnamed: 0_y']
del df['tag_nr']
del df['tag_length']
del df['tag_mcs']
del df['tag_TXbf']
del df['tag_Antsel']
del df['is_valid']


In [252]:
df.head()

,epoch_ts,frame_nr,mac_address,rssi,seq_num,ht_capabilities,ht_ampdu,tag_HTex,latitude,longitude,speed_over_ground_knots,heading_true_course,line,bus,Number_of_Passengers
0,2021-04-12 09:22:24,1,d6:26:b4:b7:b0:93,-57,2030,0x0000016f,0x00000013,0x00000000,48.151754,11.525763,19.709,259.68,trip_1,bus_trip_1,-1.0
1,2021-04-12 09:22:24,2,d6:26:b4:b7:b0:93,-59,2031,0x0000016f,0x00000013,0x00000000,48.151754,11.525763,19.709,259.68,trip_1,bus_trip_1,-1.0
2,2021-04-12 09:22:28,3,92:6a:96:38:22:e7,-43,2083,0x0000402d,0x00000017,0x00000000,48.151676,11.525196,19.985,260.49,trip_1,bus_trip_1,-1.0
3,2021-04-12 09:22:28,4,92:6a:96:38:22:e7,-43,2084,0x0000402d,0x00000017,0x00000000,48.151676,11.525196,19.985,260.49,trip_1,bus_trip_1,-1.0
4,2021-04-12 09:22:30,8,ec:10:7b:6b:21:fc,-35,1447,0x0000016e,0x00000003,0x00000000,48.151616,11.524912,20.281,258.78,trip_1,bus_trip_1,12.0


In [253]:
def mac_to_int(mac_str):
    # takes mac address as tring and converts to int
    return int(''.join(mac_str.split(":")), 16)

In [254]:
def int_to_mac(mac_int):
    mac_hex = "{:012x}".format(mac_int)
    mac_str = ":".join(mac_hex[i:i+2] for i in range(0, len(mac_hex), 2))
    return mac_str

In [255]:
# df['mac_address'] = mac_to_int(str(df['mac_address']))

In [256]:
addresses = list(df['mac_address'])
for i in df.index:
    df.at[i, 'mac_address'] = mac_to_int(addresses[i])

In [257]:
df = df.sort_values(by='epoch_ts',ascending=True)
df = df.reset_index(drop=True)
df['passengers'] = 0

In [258]:
df.head()

,epoch_ts,frame_nr,mac_address,rssi,seq_num,ht_capabilities,ht_ampdu,tag_HTex,latitude,longitude,speed_over_ground_knots,heading_true_course,line,bus,Number_of_Passengers,passengers
0,2021-04-12 09:22:24,1,235461729038483,-57,2030,0x0000016f,0x00000013,0x00000000,48.151754,11.525763,19.709,259.68,trip_1,bus_trip_1,-1.0,0
1,2021-04-12 09:22:24,2,235461729038483,-59,2031,0x0000016f,0x00000013,0x00000000,48.151754,11.525763,19.709,259.68,trip_1,bus_trip_1,-1.0,0
2,2021-04-12 09:22:28,3,160986484450023,-43,2083,0x0000402d,0x00000017,0x00000000,48.151676,11.525196,19.985,260.49,trip_1,bus_trip_1,-1.0,0
3,2021-04-12 09:22:28,4,160986484450023,-43,2084,0x0000402d,0x00000017,0x00000000,48.151676,11.525196,19.985,260.49,trip_1,bus_trip_1,-1.0,0
4,2021-04-12 09:22:30,8,259555534250492,-35,1447,0x0000016e,0x00000003,0x00000000,48.151616,11.524912,20.281,258.78,trip_1,bus_trip_1,12.0,0


In [271]:
flag = True
current_station = set()
next_station = set() 
count = 0
time = 0
speed_flag = False
start_index = 0
for index, row in df.iterrows():
    if flag:
        current_station.add(row['mac_address'])
    else:
        next_station.add(row['mac_address'])
    if row['speed_over_ground_knots'] < 0.2:
        speed_flag = True
    if row['speed_over_ground_knots'] > 3 and speed_flag:
        count += 1
        flag = False
        speed_flag = False
    if count == 2:
        flag = True
        count = 0
#         print(current_station)
#         print(next_station)
#         intersection = set.intersection(current_station, next_station)
#         print(intersection)
#         break
        intersection = set.intersection(current_station, next_station)
        passenger_count = len(intersection)
#         print(passenger_count)
        df.loc[start_index:index, 'passengers'] = passenger_count
        start_index = index
        current_station = set()
        next_station = set()


In [273]:
# df.loc[[0,48], 'passengers'] = 15
df.tail(50)

,epoch_ts,frame_nr,mac_address,rssi,seq_num,ht_capabilities,ht_ampdu,tag_HTex,latitude,longitude,speed_over_ground_knots,heading_true_course,line,bus,Number_of_Passengers,passengers
89454,2021-04-15 19:01:06,3721,202993197427882,-83,3244,0x00000162,0x00000003,0x00000000,48.151844,11.536404,0.039,NaN,trip_12,bus_trip_12,-1.0,3
89455,2021-04-15 19:01:08,3722,86387547721225,-79,3197,0x000001ae,0x0000001b,0x00000000,48.151846,11.536395,1.026,NaN,trip_12,bus_trip_12,-1.0,3
89456,2021-04-15 19:01:09,3725,175972241635055,-67,304,0x0000186c,0x0000001a,0x00000000,48.151844,11.536378,2.486,269.07,trip_12,bus_trip_12,-1.0,3
89457,2021-04-15 19:01:09,3724,268706565456363,-75,3038,0x000001ad,0x0000001b,0x00000000,48.151844,11.536378,2.486,269.07,trip_12,bus_trip_12,-1.0,3
89458,2021-04-15 19:01:09,3723,175972241635055,-69,303,0x0000186c,0x0000001a,0x00000000,48.151844,11.536378,2.486,269.07,trip_12,bus_trip_12,-1.0,3
89459,2021-04-15 19:01:15,3730,204718218564481,-53,1483,0x000001ad,0x00000013,0x00000000,48.151934,11.536041,10.222,292.84,trip_12,bus_trip_12,-1.0,3
89460,2021-04-15 19:01:15,3731,204718218564481,-53,1484,0x000001ad,0x00000013,0x00000000,48.151934,11.536041,10.222,292.84,trip_12,bus_trip_12,-1.0,0
89461,2021-04-15 19:01:15,3732,204718218564481,-53,1485,0x000001ad,0x00000013,0x00000000,48.151934,11.536041,10.222,292.84,trip_12,bus_trip_12,-1.0,0
89462,2021-04-15 19:01:15,3733,204718218564481,-55,1488,0x000001ad,0x00000013,0x00000000,48.151934,11.536041,10.222,292.84,trip_12,bus_trip_12,-1.0,0
89463,2021-04-15 19:01:15,3734,204718218564481,-55,1489,0x000001ad,0x00000013,0x00000000,48.151934,11.536041,10.222,292.84,trip_12,bus_trip_12,-1.0,0


In [274]:
df.to_csv('df.csv')

In [237]:
df.index

RangeIndex(start=0, stop=89504, step=1)

In [189]:
df['mac_address'][1]

235461729038483